# Краудфандиг на Kickstarter

## Загрузка данных

In [56]:
# Импортируем библиотеки

import pandas as pd
import numpy as np

In [57]:
# Загружаем данные в dataframe

data = pd.read_csv('data/ks.csv')

In [58]:
data.head()

,Название,Категория,Главная категория,Валюта,Дедлайн,Дата публикации,Состояние,Инвесторов,Страна,Собрано в долларах,Цель в долларах
0,"Don't Call it a Comeback ""Telescopes""",Music,Music,USD,2013-01-10,2012-12-09 06:03:52,successful,23,US,600.00,600.00
1,Arcade County (Canceled),Games,Games,USD,2012-04-29,2012-03-30 23:40:45,canceled,5,US,71.00,9000.00
2,Hayashi Skate Co. Solar Skateboard backpack,Accessories,Fashion,CAD,2017-07-22,2017-05-23 23:00:13,canceled,8,CA,360.36,2391.77
3,Me & You Coordinating Sunglasses- Optical Qual...,Accessories,Fashion,USD,2016-11-18,2016-10-19 22:06:41,failed,20,US,502.00,10000.00
4,New Carts for Istanbul Street Food Vendors,Food,Food,USD,2015-05-17,2015-04-17 18:10:47,successful,62,US,2414.00,1400.00


In [59]:
data.shape

(378661, 11)

In [60]:
### Проверим распределение проектов по признаку "Состояние"

data['Состояние'].value_counts()

Состояние
failed        197719
successful    133956
canceled       38779
undefined       3562
live            2799
suspended       1846
Name: count, dtype: int64

In [61]:
### Оставим только проекты с состоянием failed и successful

data = data[data['Состояние'].isin(['failed','successful'])]

### 2 варианта таргета:

- Классификация. Те проекты, у которых успешный статус, единичкой, а остальные ноликами

- Регрессия. Можем предсказывать просто собранное количество денег, применять модель, а потом уже смотреть, нужная ли сумма получилось.

In [62]:
data['Состояние'].value_counts()

Состояние
failed        197719
successful    133956
Name: count, dtype: int64

In [63]:
### Создадим колонку target1 (бинарный таргет: 0 - неуспешный проект, 1 - успешный проект)

# data.loc[(data['Состояние'] == 'failed'), 'таргет1'] = 0
# data['таргет1'] = data['таргет1'].fillna(1)

data['таргет1'] = data['Состояние'].apply(lambda x: 0 if x=='failed' else 1) 

In [64]:
data.head()

,Название,Категория,Главная категория,Валюта,Дедлайн,Дата публикации,Состояние,Инвесторов,Страна,Собрано в долларах,Цель в долларах,таргет1
0,"Don't Call it a Comeback ""Telescopes""",Music,Music,USD,2013-01-10,2012-12-09 06:03:52,successful,23,US,600.00,600.0,1
3,Me & You Coordinating Sunglasses- Optical Qual...,Accessories,Fashion,USD,2016-11-18,2016-10-19 22:06:41,failed,20,US,502.00,10000.0,0
4,New Carts for Istanbul Street Food Vendors,Food,Food,USD,2015-05-17,2015-04-17 18:10:47,successful,62,US,2414.00,1400.0,1
5,New Improv Comedy Venue in Des Moines,Theater,Theater,USD,2013-06-17,2013-05-03 16:17:21,successful,86,US,10030.88,10000.0,1
6,The Seer and the Sword,Shorts,Film & Video,USD,2012-08-11,2012-07-12 05:19:53,failed,0,US,0.00,10000.0,0


In [65]:
### Удалим столбец состояние

data = data.drop('Состояние', axis = 1)

In [66]:
### Создадим вещественный таргет2 (сколько денег соберет проект)

data = data.rename({'Собрано в долларах': 'таргет2'}, axis = 1)
data.head()

,Название,Категория,Главная категория,Валюта,Дедлайн,Дата публикации,Инвесторов,Страна,таргет2,Цель в долларах,таргет1
0,"Don't Call it a Comeback ""Telescopes""",Music,Music,USD,2013-01-10,2012-12-09 06:03:52,23,US,600.00,600.0,1
3,Me & You Coordinating Sunglasses- Optical Qual...,Accessories,Fashion,USD,2016-11-18,2016-10-19 22:06:41,20,US,502.00,10000.0,0
4,New Carts for Istanbul Street Food Vendors,Food,Food,USD,2015-05-17,2015-04-17 18:10:47,62,US,2414.00,1400.0,1
5,New Improv Comedy Venue in Des Moines,Theater,Theater,USD,2013-06-17,2013-05-03 16:17:21,86,US,10030.88,10000.0,1
6,The Seer and the Sword,Shorts,Film & Video,USD,2012-08-11,2012-07-12 05:19:53,0,US,0.00,10000.0,0


### Обработаем сырые данные. Извлечем признаки, где они не даны явно.

In [67]:
### Оставим от колонок с датами разницу в днях

data['Дедлайн'] = pd.to_datetime(data['Дедлайн'])
data['Дата публикации'] = pd.to_datetime(data['Дата публикации'])

In [68]:
data['Срок'] = (data['Дедлайн'] - data['Дата публикации']).dt.days

In [69]:
### Добавим Год публикации

data['Год публикации'] = data['Дата публикации'].dt.year

In [70]:
data.head()

,Название,Категория,Главная категория,Валюта,Дедлайн,Дата публикации,Инвесторов,Страна,таргет2,Цель в долларах,таргет1,Срок,Год публикации
0,"Don't Call it a Comeback ""Telescopes""",Music,Music,USD,2013-01-10,2012-12-09 06:03:52,23,US,600.00,600.0,1,31,2012
3,Me & You Coordinating Sunglasses- Optical Qual...,Accessories,Fashion,USD,2016-11-18,2016-10-19 22:06:41,20,US,502.00,10000.0,0,29,2016
4,New Carts for Istanbul Street Food Vendors,Food,Food,USD,2015-05-17,2015-04-17 18:10:47,62,US,2414.00,1400.0,1,29,2015
5,New Improv Comedy Venue in Des Moines,Theater,Theater,USD,2013-06-17,2013-05-03 16:17:21,86,US,10030.88,10000.0,1,44,2013
6,The Seer and the Sword,Shorts,Film & Video,USD,2012-08-11,2012-07-12 05:19:53,0,US,0.00,10000.0,0,29,2012


### Добавим дополнительные данные для объяснения таргет переменной

In [71]:
### Добавим макропоказатели

Macro = pd.read_excel("data/macrofeatures.xlsx", engine = "openpyxl")

In [72]:
Macro.head()

,Unnamed: 0,Close_brent,Close_sugar,Close_cereals,Close_index_moex,Close_index_moex_10,Close_index_RGBI,Close_index_RTS_oil_and_gas,Close_index_RTS_metallurgy,Close_index_RTS_consumer_sector,Close_index_RTS_telecom,Close_index_RTS_finance,Close_index_RTS_transport,Close_index_RTS_chemicals,Close_index_RTS_broad_market,Close_index_RTS_electricity,dlk_cob_date
0,0,34.41,13.97,442.75,1797.27,3940.81,125.59,123.40,111.97,196.55,70.17,140.57,27.06,177.38,530.59,32.49,2016-02-24
1,1,35.06,14.24,445.25,1803.89,3977.35,126.44,124.22,112.51,198.03,70.56,142.64,27.43,179.48,536.20,33.07,2016-02-25
2,2,35.13,14.00,443.25,1816.73,4027.23,126.90,125.38,113.44,200.13,71.94,145.45,28.06,181.56,544.73,33.55,2016-02-26
3,3,36.64,14.36,445.00,1840.17,4084.24,126.87,126.69,114.66,200.32,72.41,147.22,28.49,186.76,552.82,34.41,2016-02-29
4,4,36.60,14.39,438.50,1844.17,4087.06,127.78,129.72,117.09,204.30,74.26,150.04,30.12,190.67,565.45,34.96,2016-03-01


In [73]:
### Смёрджим колонки, оставив одну макрофичу (цену закрытия нефти)

Macro = Macro[['Close_brent', 'dlk_cob_date']].drop_duplicates()

In [74]:
Macro.groupby('dlk_cob_date')['Close_brent'].count()

dlk_cob_date
2016-02-24    1
2016-02-25    1
2016-02-26    1
2016-02-29    1
2016-03-01    1
             ..
2021-11-22    1
2021-11-23    1
2021-11-24    1
2021-11-26    1
2021-11-29    1
Name: Close_brent, Length: 1288, dtype: int64

In [75]:
Macro['dlk_cob_date'] = pd.to_datetime(Macro['dlk_cob_date'])

In [76]:
data['Дата публикации'] = data['Дата публикации'].astype("datetime64[ns]")

In [77]:
data = pd.merge(data,
         Macro,
         left_on = "Дата публикации",
         right_on = "dlk_cob_date",
         how = "left")

In [78]:
data = data.sort_values('Дата публикации')

In [79]:
### Заполним пропущенные значения в цене нефти

data['Close_brent'] = data['Close_brent'].fillna(34.41)

### Избавляемся от колонок, не являющихся признаками

In [80]:
### Уберем колонки Дедлайн, Дата публикации, dlk_cob_date

data = data.drop(['Дедлайн', 'Дата публикации', 'dlk_cob_date'], axis = 1)
data.head()

,Название,Категория,Главная категория,Валюта,Инвесторов,Страна,таргет2,Цель в долларах,таргет1,Срок,Год публикации,Close_brent
176128,Grace Jones Does Not Give A F$#% T-Shirt (limi...,Fashion,Fashion,USD,30,US,625.0,1000.0,0,39,2009,34.41
241929,CRYSTAL ANTLERS UNTITLED MOVIE,Shorts,Film & Video,USD,3,US,22.0,80000.0,0,87,2009,34.41
244460,drawing for dollars,Illustration,Art,USD,3,US,35.0,20.0,1,8,2009,34.41
80845,Offline Wikipedia iPhone app,Software,Technology,USD,25,US,145.0,99.0,1,79,2009,34.41
181197,Pantshirts,Fashion,Fashion,USD,10,US,387.0,1900.0,0,28,2009,34.41


In [81]:
### Уберем признаки: Название (так как оно как id), Страна (Данные уже представлены в колонке Валюта), Инвесторов (так как мы можем посчитать только постфактум)

data = data.drop(['Название', 'Страна', 'Инвесторов'], axis = 1)
data.head()

,Категория,Главная категория,Валюта,таргет2,Цель в долларах,таргет1,Срок,Год публикации,Close_brent
176128,Fashion,Fashion,USD,625.0,1000.0,0,39,2009,34.41
241929,Shorts,Film & Video,USD,22.0,80000.0,0,87,2009,34.41
244460,Illustration,Art,USD,35.0,20.0,1,8,2009,34.41
80845,Software,Technology,USD,145.0,99.0,1,79,2009,34.41
181197,Fashion,Fashion,USD,387.0,1900.0,0,28,2009,34.41


### One-hot encoding

In [82]:
### One-hot encoding колонки с валютой

data = pd.concat((data, pd.get_dummies(data['Валюта']).astype(int)), axis=1)
data = data.drop(['Валюта'], axis = 1)

In [83]:
data.head()

,Категория,Главная категория,таргет2,Цель в долларах,таргет1,Срок,Год публикации,Close_brent,AUD,CAD,...,EUR,GBP,HKD,JPY,MXN,NOK,NZD,SEK,SGD,USD
176128,Fashion,Fashion,625.0,1000.0,0,39,2009,34.41,0,0,...,0,0,0,0,0,0,0,0,0,1
241929,Shorts,Film & Video,22.0,80000.0,0,87,2009,34.41,0,0,...,0,0,0,0,0,0,0,0,0,1
244460,Illustration,Art,35.0,20.0,1,8,2009,34.41,0,0,...,0,0,0,0,0,0,0,0,0,1
80845,Software,Technology,145.0,99.0,1,79,2009,34.41,0,0,...,0,0,0,0,0,0,0,0,0,1
181197,Fashion,Fashion,387.0,1900.0,0,28,2009,34.41,0,0,...,0,0,0,0,0,0,0,0,0,1


In [84]:
### Избавимся от избыточной информации

data = data.drop(['AUD'], axis = 1)

In [85]:
### One-hot encoding колонки с главной категорией

data = pd.concat((data, pd.get_dummies(data['Главная категория']).astype(int)), axis=1)
data = data.drop(['Главная категория'], axis = 1)
data.head()

,Категория,таргет2,Цель в долларах,таргет1,Срок,Год публикации,Close_brent,CAD,CHF,DKK,...,Fashion,Film & Video,Food,Games,Journalism,Music,Photography,Publishing,Technology,Theater
176128,Fashion,625.0,1000.0,0,39,2009,34.41,0,0,0,...,1,0,0,0,0,0,0,0,0,0
241929,Shorts,22.0,80000.0,0,87,2009,34.41,0,0,0,...,0,1,0,0,0,0,0,0,0,0
244460,Illustration,35.0,20.0,1,8,2009,34.41,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80845,Software,145.0,99.0,1,79,2009,34.41,0,0,0,...,0,0,0,0,0,0,0,0,1,0
181197,Fashion,387.0,1900.0,0,28,2009,34.41,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [86]:
data = data.drop(['Games'], axis = 1)

In [87]:
### Посмотрим на количество вариантов в столбце Категория
data['Категория'].unique()

array(['Fashion', 'Shorts', 'Illustration', 'Software', 'Journalism',
       'Fiction', 'Rock', 'Photography', 'Puzzles', 'Graphic Design',
       'Film & Video', 'Publishing', 'Documentary', 'Theater',
       'Sculpture', 'Electronic Music', 'Nonfiction', 'Painting',
       'Indie Rock', 'Public Art', 'Art', 'Crafts', 'Jazz', 'Music',
       'Comics', "Children's Books", 'Narrative Film', 'Tabletop Games',
       'Video Games', 'Digital Art', 'Food', 'Animation',
       'Conceptual Art', 'Pop', 'Hip-Hop', 'Country & Folk',
       'Periodicals', 'Webseries', 'Product Design', 'Performance Art',
       'Art Books', 'World Music', 'Knitting', 'Technology',
       'Classical Music', 'Graphic Novels', 'Poetry', 'Radio & Podcasts',
       'Design', 'Hardware', 'Webcomics', 'Dance', 'Translations',
       'Crochet', 'Games', 'Photo', 'Mixed Media', 'Space Exploration',
       'Photobooks', 'Musical', 'Audio', 'Community Gardens', 'R&B',
       'Fabrication Tools', 'Textiles', 'Architecture',

### Mean-target encoding (Счётчики)

In [88]:
### Проведем mean-target encoding колонки Категория

data['Категория'] = data['Категория'].map(data.groupby(['Категория'])['таргет2'].mean())

In [89]:
data.head()

,Категория,таргет2,Цель в долларах,таргет1,Срок,Год публикации,Close_brent,CAD,CHF,DKK,...,Design,Fashion,Film & Video,Food,Journalism,Music,Photography,Publishing,Technology,Theater
176128,6035.989239,625.0,1000.0,0,39,2009,34.41,0,0,0,...,0,1,0,0,0,0,0,0,0,0
241929,3591.033473,22.0,80000.0,0,87,2009,34.41,0,0,0,...,0,0,1,0,0,0,0,0,0,0
244460,3661.424550,35.0,20.0,1,8,2009,34.41,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80845,4321.245721,145.0,99.0,1,79,2009,34.41,0,0,0,...,0,0,0,0,0,0,0,0,1,0
181197,6035.989239,387.0,1900.0,0,28,2009,34.41,0,0,0,...,0,1,0,0,0,0,0,0,0,0


## Определимся с таргетом

In [90]:
### Так как будем решать задачу регрессии, уберем target1

data = data.drop("таргет1", axis=1)

In [91]:
### Разделим данные на объекты и ответы

X = data.drop('таргет2', axis=1)
Y = data['таргет2']

In [92]:
X

,Категория,Цель в долларах,Срок,Год публикации,Close_brent,CAD,CHF,DKK,EUR,GBP,...,Design,Fashion,Film & Video,Food,Journalism,Music,Photography,Publishing,Technology,Theater
176128,6035.989239,1000.00,39,2009,34.41,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
241929,3591.033473,80000.00,87,2009,34.41,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
244460,3661.424550,20.00,8,2009,34.41,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80845,4321.245721,99.00,79,2009,34.41,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
181197,6035.989239,1900.00,28,2009,34.41,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247730,7635.064778,35.98,2,2017,34.41,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
318187,38415.722876,271.03,4,2017,34.41,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
264474,6098.303122,200.00,3,2017,34.41,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
293634,38415.722876,250.00,1,2017,34.41,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
Y

176128    625.00
241929     22.00
244460     35.00
80845     145.00
181197    387.00
           ...  
247730    269.84
318187    544.77
264474    202.00
293634    265.00
23315     513.60
Name: таргет2, Length: 331675, dtype: float64

## Scikit-learn. Модель линейной регресии

In [94]:
from sklearn.linear_model import LinearRegression

In [95]:
### Обучим модель линейной регрессии

model = LinearRegression()
model.fit(X, Y)

X['Предсказание'] = model.predict(X)

X.head()

,Категория,Цель в долларах,Срок,Год публикации,Close_brent,CAD,CHF,DKK,EUR,GBP,...,Fashion,Film & Video,Food,Journalism,Music,Photography,Publishing,Technology,Theater,Предсказание
176128,6035.989239,1000.0,39,2009,34.41,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,3125.804988
241929,3591.033473,80000.0,87,2009,34.41,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,5117.789402
244460,3661.424550,20.0,8,2009,34.41,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1680.580057
80845,4321.245721,99.0,79,2009,34.41,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,4935.864467
181197,6035.989239,1900.0,28,2009,34.41,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,2183.699047


In [96]:
data.head()

,Категория,таргет2,Цель в долларах,Срок,Год публикации,Close_brent,CAD,CHF,DKK,EUR,...,Design,Fashion,Film & Video,Food,Journalism,Music,Photography,Publishing,Technology,Theater
176128,6035.989239,625.0,1000.0,39,2009,34.41,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
241929,3591.033473,22.0,80000.0,87,2009,34.41,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
244460,3661.424550,35.0,20.0,8,2009,34.41,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80845,4321.245721,145.0,99.0,79,2009,34.41,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
181197,6035.989239,387.0,1900.0,28,2009,34.41,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


### MSE

In [97]:
### Посчитаем MSE

X['Предсказание'] - Y

176128     2500.804988
241929     5095.789402
244460    -1715.580057
80845      4790.864467
181197     1796.699047
              ...     
247730     2050.451701
318187    34795.145211
264474     6940.933046
293634    39056.799216
23315     -5217.684006
Length: 331675, dtype: float64

In [98]:
((X['Предсказание'] - Y)**2).mean()

np.float64(9201867212.575926)

## RMSE

In [99]:
### RMSE

(((X['Предсказание'] - Y)**2).mean()) ** 0.5

np.float64(95926.36349083565)

### MAE

In [100]:
### Посчитаем MAE

abs(X['Предсказание'] - Y).mean()

np.float64(13854.018404845567)

## Подбор параметров через матричный метод

In [101]:
X = X.drop("Предсказание", axis = 1)

In [102]:
X.head()

,Категория,Цель в долларах,Срок,Год публикации,Close_brent,CAD,CHF,DKK,EUR,GBP,...,Design,Fashion,Film & Video,Food,Journalism,Music,Photography,Publishing,Technology,Theater
176128,6035.989239,1000.0,39,2009,34.41,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
241929,3591.033473,80000.0,87,2009,34.41,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
244460,3661.424550,20.0,8,2009,34.41,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80845,4321.245721,99.0,79,2009,34.41,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
181197,6035.989239,1900.0,28,2009,34.41,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [103]:
Y.head()

176128    625.0
241929     22.0
244460     35.0
80845     145.0
181197    387.0
Name: таргет2, dtype: float64

In [104]:
### Посмотрим на параметры линейной регрессии

model = LinearRegression()
model.fit(X, Y)

for column, coef in zip(X.columns, model.coef_):
    print(column, coef)

print(model.intercept_)

Категория 0.9918420191805425
Цель в долларах 0.0004601639134073805
Срок 85.68364438055293
Год публикации 880.0377324248398
Close_brent -291.70539947688303
CAD 473.78978615222456
CHF 8893.788845666732
DKK 542.937077575855
EUR 480.5792834985892
GBP 2799.1772631243925
HKD -2065.4276576211414
JPY -8469.152380339456
MXN -3973.272543141307
NOK -1161.8153373469102
NZD -382.4190440448499
SEK 2118.1644133462787
SGD -1319.8569240200575
USD 7038.121878302784
Art -90.9836589513418
Comics -506.47356177510056
Crafts -629.9052679698799
Dance -366.1570059824404
Design -232.50569157168047
Fashion -296.435586737362
Film & Video -28.609188313929153
Food -724.8630732575516
Journalism -475.29840480275226
Music -273.49063202842615
Photography 220.82017844714656
Publishing -362.7085389636493
Technology -212.55260184943162
Theater 260.13217628445716
-1770902.9729986964


Теперь реализуем функцию LinearRegressionByMatrix, которая будет принимать на вход 3 параметра:

Матрицу объект-признак **(X)**, вектор с ответами **(Y)**, булевый параметр **fit_intercept**, цель которого - добавить константный признак (состоящий из единичек), если True, и ничего не делать, если False.

Функция должна вернуть одномерный np.array объект с оцененными **$\beta_1, ..., \beta_n, \beta_0$**

Реализуем решение через аналитическое решение в матрицах:

$$
\beta^* = (X^T \cdot X)^{-1} \cdot X^T \cdot Y
$$

In [105]:
def LinearRegressionParametersByMatrix(X: np.array, Y: np.array, fit_intercept: bool = True):
        """
        :param X: матрица объектов
        :param Y: вектор (матрица с 1 столбцом) ответов
        :param fit_intercept: добавить ли константную переменную в данные?
        
        :return: одномерный numpy-array с полученными коэффициентами beta
        """

        if fit_intercept:
                ones = np.ones((X.shape[0], 1))
                X = np.hstack((X, ones))
        
        XTX = np.dot(X.T, X)
        XTX_inv = np.linalg.inv(XTX)
        XTX_inv_XT = np.dot(XTX_inv, X.T)
        final_betas = np.dot(XTX_inv_XT, Y)

        return final_betas

In [106]:
LinearRegressionParametersByMatrix(X, Y)

array([ 9.91842019e-01,  4.60163914e-04,  8.56836441e+01,  8.80037732e+02,
       -2.91702786e+02,  4.73789786e+02,  8.89378885e+03,  5.42937078e+02,
        4.80579282e+02,  2.79917726e+03, -2.06542766e+03, -8.46915238e+03,
       -3.97327254e+03, -1.16181534e+03, -3.82419044e+02,  2.11816441e+03,
       -1.31985692e+03,  7.03812188e+03, -9.09836593e+01, -5.06473562e+02,
       -6.29905268e+02, -3.66157007e+02, -2.32505692e+02, -2.96435589e+02,
       -2.86091890e+01, -7.24863073e+02, -4.75298405e+02, -2.73490633e+02,
        2.20820178e+02, -3.62708540e+02, -2.12552602e+02,  2.60132176e+02,
       -1.77090306e+06])